In [1]:
# set workspace
WhoIsWorking = "RT"
WorkLocation = "School"

if WhoIsWorking == "RT":
    if WorkLocation == "Home":
        folder_path = r'C:\Users\rentr\Desktop\ENGO\DataMining\ENGO645_TermProject_Data'
    elif WorkLocation == "School":
        folder_path = r'D:\ENGO645_TermProject_Data'
elif WhoIsWorking == "AA":
    folder_path = ''
elif WhoIsWorking == "IA":
    folder_path = r'C:\Users\afoam\assignments\ENGO645_TorontoCrimeAnalysis'
elif WhoIsWorking == "AI":
    folder_path = r'C:\Adewale_Directory\Capacity_Building\MGIS\Lecture_Note\WINTER2024\ENGO645\Course_Project\Project_Data'

print(folder_path)

D:\ENGO645_TermProject_Data


In [2]:
#Install and import neccessary libraries
import os
import numpy as np
import geopandas as gpd
import pandas as pd
from libpysal.weights import lat2W
from esda.moran import Moran

In [3]:

# Load and read the MCI file
MCI_file = os.path.join(folder_path, "MCI_Cleaned.csv") # Update to the actual file location
MCI_data = pd.read_csv(MCI_file)
 
MCI_data #display dataframe

,OCC_YEAR,OCC_MONTH,OCC_DAY,OCC_DOY,OCC_DOW,OCC_HOUR,LOCATION_TYPE,PREMISES_TYPE,OFFENCE,MCI_CATEGORY,NEIGHBOURHOOD_158,LONG_WGS84,LAT_WGS84,DATE_TIME_LINK,TIMECategory,TEMP,TEMPCategory
0,2023,January,1,1,Sunday,2,"Streets, Roads, Highways (Bicycle Path, Privat...",Outside,Robbery With Weapon,Robbery,Corso Italia-Davenport,-79.450529,43.676205,2023/1/1 2,Night,4.7,Mild
1,2023,January,1,1,Sunday,1,"Streets, Roads, Highways (Bicycle Path, Privat...",Outside,Robbery - Other,Robbery,St Lawrence-East Bayfront-The Islands,-79.368457,43.648776,2023/1/1 1,Night,4.5,Mild
2,2023,January,1,1,Sunday,9,"Apartment (Rooming House, Condo)",Apartment,Assault,Assault,Morningside Heights,-79.202638,43.815274,2023/1/1 9,Morning,4.4,Mild
3,2023,January,1,1,Sunday,3,"Streets, Roads, Highways (Bicycle Path, Privat...",Outside,Assault With Weapon,Assault,New Toronto,-79.517052,43.600915,2023/1/1 3,Night,4.9,Mild
4,2023,January,1,1,Sunday,21,"Streets, Roads, Highways (Bicycle Path, Privat...",Outside,Theft Of Motor Vehicle,Auto Theft,Victoria Village,-79.300571,43.719703,2023/1/1 21,Evening,4.1,Mild
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
365681,2014,December,31,365,Wednesday,16,"Apartment (Rooming House, Condo)",Apartment,B&E,Break and Enter,Wellington Place,-79.390275,43.647849,2014/12/31 16,Afternoon,-6.3,Chilly
365682,2014,December,31,365,Wednesday,12,"Single Home, House (Attach Garage, Cottage, Mo...",House,Assault With Weapon,Assault,Bendale-Glen Andrew,-79.248690,43.761776,2014/12/31 12,Afternoon,-6.3,Chilly
365683,2014,December,31,365,Wednesday,12,"Single Home, House (Attach Garage, Cottage, Mo...",House,Assault,Assault,Bendale-Glen Andrew,-79.248690,43.761776,2014/12/31 12,Afternoon,-6.3,Chilly
365684,2014,December,31,365,Wednesday,22,"Apartment (Rooming House, Condo)",Apartment,Assault,Assault,Greenwood-Coxwell,-79.325929,43.671381,2014/12/31 22,Evening,-5.7,Chilly


In [4]:
# Find the count of offenses by neighborhood to aggregate data to Neighbourhood level
neighbourhood_counts = MCI_data['NEIGHBOURHOOD_158'].value_counts().reset_index()
neighbourhood_counts.columns = ['NeighbourhoodName', 'Crime_Count']  # Rename columns

neighbourhood_counts

,NeighbourhoodName,Crime_Count
0,West Humber-Clairville,10309
1,Moss Park,8621
2,Downtown Yonge East,7762
3,Yonge-Bay Corridor,7294
4,York University Heights,7237
...,...,...
154,Maple Leaf,734
155,Guildwood,680
156,Woodbine-Lumsden,649
157,Lambton Baby Point,585


In [5]:
# Load and read the Neighbourhood file
neighbourhood_file = os.path.join(folder_path, "Neighbourhood_Cleaned.csv")
neighbourhood_data = pd.read_csv(neighbourhood_file)

In [6]:
# Join the aggregated crime counts with neighborhood data based on the common neighborhood field
neighbourhood_merged = pd.merge(neighbourhood_data, neighbourhood_counts, on='NeighbourhoodName', how='left')

# Drop rows with missing values (if any)
neighbourhood_merged = neighbourhood_merged.dropna(subset=['Crime_Count'])

In [7]:
# Check the shape and info of the data again
print(neighbourhood_merged.shape)
neighbourhood_merged.info()

(158, 12)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 158 entries, 0 to 157
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   NeighbourhoodName           158 non-null    object 
 1   Poles_Count                 158 non-null    int64  
 2   POI_Count                   158 non-null    int64  
 3   TCamera_Count               158 non-null    int64  
 4   TotalPopulation             158 non-null    int64  
 5   MedianTotalIncome           158 non-null    int64  
 6   AverageTotalIncome          158 non-null    int64  
 7   BuildingCoveragePERCENTAGE  158 non-null    float64
 8   ParksOSNAPERCENTAGE         158 non-null    float64
 9   BuiltUpPERCENTAGE           158 non-null    float64
 10  AveragePrice                158 non-null    int64  
 11  Crime_Count                 158 non-null    int64  
dtypes: float64(3), int64(8), object(1)
memory usage: 14.9+ KB


In [8]:
# Check Neighbourhood File
# Load the Neighbourhood file
Neighbourhood_file = os.path.join(folder_path, "Neighbourhood_Cleaned.csv")
Neighbourhood_data = pd.read_csv(Neighbourhood_file)

# Drop the 'geometry' column
if 'geometry' in Neighbourhood_data.columns:
    Neighbourhood_data.drop(columns=['geometry'], inplace=True)
else:
    print("Column 'geometry' not found in the DataFrame.")

Neighbourhood_data

Column 'geometry' not found in the DataFrame.


,NeighbourhoodName,Poles_Count,POI_Count,TCamera_Count,TotalPopulation,MedianTotalIncome,AverageTotalIncome,BuildingCoveragePERCENTAGE,ParksOSNAPERCENTAGE,BuiltUpPERCENTAGE,AveragePrice
0,South Eglinton-Davisville,649,1185,2,22735,52400,69000,29.087555,6.375824,93.624176,1035636
1,North Toronto,286,851,2,15885,46000,58100,31.403497,1.133288,98.866712,1035636
2,Dovercourt Village,1170,534,1,12380,38000,55850,33.231409,7.875048,92.124952,1266076
3,Junction-Wallace Emerson,1663,876,1,23180,41200,54350,33.167140,6.821523,93.178477,1266076
4,Yonge-Bay Corridor,1139,20281,12,12645,44000,68200,48.028587,6.382056,93.617944,880955
...,...,...,...,...,...,...,...,...,...,...,...
153,West Humber-Clairville,7704,4254,2,33300,33600,40560,17.465098,11.373365,88.626635,811543
154,Black Creek,1148,541,1,21145,29400,33720,17.667624,15.911150,84.088850,836734
155,Pelmo Park-Humberlea,1223,369,1,11290,39600,49000,14.734703,17.357255,82.642745,836734
156,Humbermede,1031,707,1,16955,32400,38280,17.920562,21.469614,78.530386,836734


In [9]:
# Load the Neighbourhood shop
NeighbourhoodBoundary_SHP = os.path.join(folder_path, "Neighbourhoods.shp")
Neighbourhood_SHP = gpd.read_file(NeighbourhoodBoundary_SHP)

# Drop all fields except geometry from Neighbourhood_SHP
Neighbourhood_SHP = Neighbourhood_SHP[['geometry', 'AREA_NA7']]

Neighbourhood_SHP

,geometry,AREA_NA7
0,"POLYGON ((-79.38635 43.69783, -79.38623 43.697...",South Eglinton-Davisville
1,"POLYGON ((-79.39744 43.70693, -79.39837 43.706...",North Toronto
2,"POLYGON ((-79.43411 43.66015, -79.43537 43.659...",Dovercourt Village
3,"POLYGON ((-79.43870 43.66766, -79.43841 43.666...",Junction-Wallace Emerson
4,"POLYGON ((-79.38404 43.64497, -79.38502 43.644...",Yonge-Bay Corridor
...,...,...
153,"POLYGON ((-79.59037 43.73401, -79.58942 43.732...",West Humber-Clairville
154,"POLYGON ((-79.51915 43.77399, -79.51901 43.773...",Black Creek
155,"POLYGON ((-79.53225 43.73505, -79.52938 43.723...",Pelmo Park-Humberlea
156,"POLYGON ((-79.52813 43.74425, -79.52721 43.740...",Humbermede


In [10]:
# Join the CSV to the shapefile using 'NeighbourhoodName' from CSV and 'AREA_NA7' from shapefile
joined_data = Neighbourhood_SHP.merge(Neighbourhood_data, left_on='AREA_NA7', right_on='NeighbourhoodName', how='right')

joined_data

,geometry,AREA_NA7,NeighbourhoodName,Poles_Count,POI_Count,TCamera_Count,TotalPopulation,MedianTotalIncome,AverageTotalIncome,BuildingCoveragePERCENTAGE,ParksOSNAPERCENTAGE,BuiltUpPERCENTAGE,AveragePrice
0,"POLYGON ((-79.38635 43.69783, -79.38623 43.697...",South Eglinton-Davisville,South Eglinton-Davisville,649,1185,2,22735,52400,69000,29.087555,6.375824,93.624176,1035636
1,"POLYGON ((-79.39744 43.70693, -79.39837 43.706...",North Toronto,North Toronto,286,851,2,15885,46000,58100,31.403497,1.133288,98.866712,1035636
2,"POLYGON ((-79.43411 43.66015, -79.43537 43.659...",Dovercourt Village,Dovercourt Village,1170,534,1,12380,38000,55850,33.231409,7.875048,92.124952,1266076
3,"POLYGON ((-79.43870 43.66766, -79.43841 43.666...",Junction-Wallace Emerson,Junction-Wallace Emerson,1663,876,1,23180,41200,54350,33.167140,6.821523,93.178477,1266076
4,"POLYGON ((-79.38404 43.64497, -79.38502 43.644...",Yonge-Bay Corridor,Yonge-Bay Corridor,1139,20281,12,12645,44000,68200,48.028587,6.382056,93.617944,880955
...,...,...,...,...,...,...,...,...,...,...,...,...,...
153,"POLYGON ((-79.59037 43.73401, -79.58942 43.732...",West Humber-Clairville,West Humber-Clairville,7704,4254,2,33300,33600,40560,17.465098,11.373365,88.626635,811543
154,"POLYGON ((-79.51915 43.77399, -79.51901 43.773...",Black Creek,Black Creek,1148,541,1,21145,29400,33720,17.667624,15.911150,84.088850,836734
155,"POLYGON ((-79.53225 43.73505, -79.52938 43.723...",Pelmo Park-Humberlea,Pelmo Park-Humberlea,1223,369,1,11290,39600,49000,14.734703,17.357255,82.642745,836734
156,"POLYGON ((-79.52813 43.74425, -79.52721 43.740...",Humbermede,Humbermede,1031,707,1,16955,32400,38280,17.920562,21.469614,78.530386,836734


In [11]:
# Check if all geometries are valid
all_valid = Neighbourhood_SHP.geometry.is_valid.all()

if all_valid:
    print("All geometries are valid.")
else:
    print("Some geometries are invalid.")

All geometries are valid.


In [12]:
# Join the aggregated crime counts with neighborhood data based on the common neighborhood field
neighbourhood_merged = pd.merge(joined_data, neighbourhood_counts, left_on='AREA_NA7', right_on='NeighbourhoodName', how='left')

# Drop rows with missing values (if any)
neighbourhood_merged = neighbourhood_merged.dropna(subset=['Crime_Count'])

In [13]:
neighbourhood_merged

,geometry,AREA_NA7,NeighbourhoodName_x,Poles_Count,POI_Count,TCamera_Count,TotalPopulation,MedianTotalIncome,AverageTotalIncome,BuildingCoveragePERCENTAGE,ParksOSNAPERCENTAGE,BuiltUpPERCENTAGE,AveragePrice,NeighbourhoodName_y,Crime_Count
0,"POLYGON ((-79.38635 43.69783, -79.38623 43.697...",South Eglinton-Davisville,South Eglinton-Davisville,649,1185,2,22735,52400,69000,29.087555,6.375824,93.624176,1035636,South Eglinton-Davisville,1539
1,"POLYGON ((-79.39744 43.70693, -79.39837 43.706...",North Toronto,North Toronto,286,851,2,15885,46000,58100,31.403497,1.133288,98.866712,1035636,North Toronto,1334
2,"POLYGON ((-79.43411 43.66015, -79.43537 43.659...",Dovercourt Village,Dovercourt Village,1170,534,1,12380,38000,55850,33.231409,7.875048,92.124952,1266076,Dovercourt Village,1652
3,"POLYGON ((-79.43870 43.66766, -79.43841 43.666...",Junction-Wallace Emerson,Junction-Wallace Emerson,1663,876,1,23180,41200,54350,33.167140,6.821523,93.178477,1266076,Junction-Wallace Emerson,2944
4,"POLYGON ((-79.38404 43.64497, -79.38502 43.644...",Yonge-Bay Corridor,Yonge-Bay Corridor,1139,20281,12,12645,44000,68200,48.028587,6.382056,93.617944,880955,Yonge-Bay Corridor,7294
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153,"POLYGON ((-79.59037 43.73401, -79.58942 43.732...",West Humber-Clairville,West Humber-Clairville,7704,4254,2,33300,33600,40560,17.465098,11.373365,88.626635,811543,West Humber-Clairville,10309
154,"POLYGON ((-79.51915 43.77399, -79.51901 43.773...",Black Creek,Black Creek,1148,541,1,21145,29400,33720,17.667624,15.911150,84.088850,836734,Black Creek,3527
155,"POLYGON ((-79.53225 43.73505, -79.52938 43.723...",Pelmo Park-Humberlea,Pelmo Park-Humberlea,1223,369,1,11290,39600,49000,14.734703,17.357255,82.642745,836734,Pelmo Park-Humberlea,1682
156,"POLYGON ((-79.52813 43.74425, -79.52721 43.740...",Humbermede,Humbermede,1031,707,1,16955,32400,38280,17.920562,21.469614,78.530386,836734,Humbermede,2669


In [14]:
# Convert the DataFrame to a GeoDataFrame
neighbourhood_gdf = gpd.GeoDataFrame(neighbourhood_merged, geometry='geometry')

# Print the GeoDataFrame to verify
display(neighbourhood_gdf.head())

,geometry,AREA_NA7,NeighbourhoodName_x,Poles_Count,POI_Count,TCamera_Count,TotalPopulation,MedianTotalIncome,AverageTotalIncome,BuildingCoveragePERCENTAGE,ParksOSNAPERCENTAGE,BuiltUpPERCENTAGE,AveragePrice,NeighbourhoodName_y,Crime_Count
0,"POLYGON ((-79.38635 43.69783, -79.38623 43.697...",South Eglinton-Davisville,South Eglinton-Davisville,649,1185,2,22735,52400,69000,29.087555,6.375824,93.624176,1035636,South Eglinton-Davisville,1539
1,"POLYGON ((-79.39744 43.70693, -79.39837 43.706...",North Toronto,North Toronto,286,851,2,15885,46000,58100,31.403497,1.133288,98.866712,1035636,North Toronto,1334
2,"POLYGON ((-79.43411 43.66015, -79.43537 43.659...",Dovercourt Village,Dovercourt Village,1170,534,1,12380,38000,55850,33.231409,7.875048,92.124952,1266076,Dovercourt Village,1652
3,"POLYGON ((-79.43870 43.66766, -79.43841 43.666...",Junction-Wallace Emerson,Junction-Wallace Emerson,1663,876,1,23180,41200,54350,33.167140,6.821523,93.178477,1266076,Junction-Wallace Emerson,2944
4,"POLYGON ((-79.38404 43.64497, -79.38502 43.644...",Yonge-Bay Corridor,Yonge-Bay Corridor,1139,20281,12,12645,44000,68200,48.028587,6.382056,93.617944,880955,Yonge-Bay Corridor,7294


In [15]:
# from shapely.geometry import Polygon
# import shapely.wkt

# # Iterate over each row in the DataFrame
# for index, row in neighbourhood_merged.iterrows():
#     # Convert the geometry string to a Shapely geometry object
#     geometry = shapely.wkt.loads(row['geometry'])
    
#     # Check if the geometry is valid
#     if not geometry.is_valid:
#         # If the geometry is invalid, attempt to fix it
#         geometry = geometry.buffer(0)
        
#         # Check if the geometry is now valid
#         if geometry.is_valid:
#             # If the geometry is valid after buffering, replace the original geometry
#             neighbourhood_merged.at[index, 'geometry'] = shapely.wkt.dumps(geometry)
#         else:
#             # If the geometry could not be fixed, handle the error accordingly
#             print(f"Unable to fix geometry at index {index}")


In [16]:
neighbourhood_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 158 entries, 0 to 157
Data columns (total 15 columns):
 #   Column                      Non-Null Count  Dtype   
---  ------                      --------------  -----   
 0   geometry                    158 non-null    geometry
 1   AREA_NA7                    158 non-null    object  
 2   NeighbourhoodName_x         158 non-null    object  
 3   Poles_Count                 158 non-null    int64   
 4   POI_Count                   158 non-null    int64   
 5   TCamera_Count               158 non-null    int64   
 6   TotalPopulation             158 non-null    int64   
 7   MedianTotalIncome           158 non-null    int64   
 8   AverageTotalIncome          158 non-null    int64   
 9   BuildingCoveragePERCENTAGE  158 non-null    float64 
 10  ParksOSNAPERCENTAGE         158 non-null    float64 
 11  BuiltUpPERCENTAGE           158 non-null    float64 
 12  AveragePrice                158 non-null    int64   
 13  Neighbourhoo

In [17]:
# Create spatial weights matrix (W) using lat2W method
W = lat2W(neighbourhood_gdf['geometry'])

# Convert the crime count to a numpy array
crime_count = neighbourhood_gdf['Crime_Count'].values

# Compute Moran's I
mi = Moran(crime_count, W)

# Print Moran's I statistic and p-value
print("Moran's I:", mi.I)
print("p-value:", mi.p_sim)


# Alternatively

# # Create the matrix of weights using lat2W
# W = lat2W(neighbourhood_gdf)

# # Create the Moran object
# mi = Moran(neighbourhood_gdf['Crime_Count'], W)

# # Print Moran's I results
# print(f"Moran's I: {mi.I:.4f}")
# print(f"p-value: {mi.p_norm:.4f}")
# moran_statistic = mi.I

TypeError: unsupported operand type(s) for *: 'GeometryArray' and 'int'

In [ ]:

# # Create spatial weights matrix (W) using lat2W method
# W = lat2W(neighbourhood_merged['geometry'].y, neighbourhood_merged['geometry'].x)

# # Convert the crime count to a numpy array
# crime_count = neighbourhood_merged['Crime_Count'].values

# # Compute Moran's I
# moran = Moran(crime_count, W)

# # Print Moran's I statistic
# print("Moran's I:", moran.I)

# OR

# # Create a numpy array from the crime count column
# data_array = np.array(neighbourhood_gdf['Crime_Count'])

# # Create the matrix of weights (assuming you want to use lat2W)
# w = lat2W(neighbourhood_gdf.geometry)

# # Create the Moran object
# mi = Moran(data_array, w)

# # Print Moran's I results
# print(f"Moran's I: {mi.I:.4f}")
# print(f"p-value: {mi.p_norm:.4f}")
# moran_statistic = mi.I
